In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
from dotenv import load_dotenv
import os

load_dotenv()
hf_token = os.getenv("HF_TOKEN")

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", token=hf_token)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", token=hf_token)

/home/junseokkim00/anaconda3/envs/biasdebate/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  7.00s/it]


In [2]:
device='cuda:0'
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,), eps=1e-05)
    (rotary_emb

In [4]:
feedback_prompt="Review your previous answer and find problems with your answer."
refine_prompt="Based on the problems you found, improve your answer. Please reiterate your answer, with your final answer a single numerical number, in the form \boxed{answer}."
answer_trigger_prompt='Therefore, the answer (arabic numerals) is'

In [5]:
question="Five friends eat at a fast-food chain and order the following: 5 pieces of hamburger that cost $3 each; 4 sets of French fries that cost $1.20; 5 cups of soda that cost $0.5 each; and 1 platter of spaghetti that cost $2.7. How much will each of them pay if they will split the bill equally?"

In [6]:
chat=[
    {"role": "user", "content": question}
]

In [12]:
def inference(chat, model, tokenizer, device):
    outputs=[]
    tokenized_chat = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    tokenized_chat = tokenized_chat.to(device)
    output = model.generate(tokenized_chat, max_new_tokens=512, do_sample=True)
    # output.shape
    decode_input = tokenizer.batch_decode(tokenized_chat, skip_special_tokens=True)
    decode_output = tokenizer.batch_decode(output, skip_special_tokens=True)
    for do,di in zip(decode_output, decode_input):
        outputs.append(do[len(di):])
    return outputs
    

In [8]:
initial_responses = inference([chat for _ in range(5)], model, tokenizer, device)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [11]:
feedbackss=[]
for initial_response in initial_responses:
    ir_chat = {"role": "assistant", "content": initial_response}
    feedback_chat = {"role": "user", "content": feedback_prompt}
    feedbacks = inference([chat+[ir_chat, feedback_chat] for _ in range(5)], model, tokenizer, device)
    extract_chat = {"role"}
    print(feedbacks)
    feedbackss.append(feedbacks)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


["Upon reviewing my previous answer, I noticed a potential issue. I didn't consider the fact that the friends are splitting the bill equally, which means they will also be splitting the cost of the spaghetti.\n\nThe original total cost was calculated as $25, but the spaghetti costs $2.70. When splitting the bill, each friend should only pay for their portion of the total cost, not the full cost of the spaghetti.\n\nTo correct this, I should have calculated the cost per friend as follows:\n\n- 5 pieces of hamburger cost 5 x $3 = $15.\n- 4 sets of French fries cost 4 x $1.20 = $4.80.\n- 5 cups of soda cost 5 x $0.5 = $2.50.\n- The spaghetti is split among the 5 friends, so each friend pays $2.70 ÷ 5 = $0.54.\n\nTotal cost per friend = $15 + $4.80 + $2.50 + $0.54 = $23.04.\n\nHowever, this is not the correct answer. The spaghetti was already included in the initial calculation of $25.\n\nA better approach would be to calculate the total cost per friend without including the spaghetti, and

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['Upon reviewing my previous answer, I notice a few potential issues:\n\n1. I did not round the costs of the items to two decimal places, which might be necessary for a real-world scenario. However, in this case, the costs are already given in two decimal places.\n\n2. I did not check if the costs of the items could be simplified or reduced before adding them together. For example, $4.80 can be simplified to $4.80.\n\n3. I did not consider the possibility that the costs might be rounded to the nearest cent in the real world, which could affect the final answer.\n\n4. I did not verify if the total cost can be simplified or reduced before dividing it by 5.\n\nLet me re-evaluate the costs:\n\n- 5 pieces of hamburger: 5 * $3 = $15\n- 4 sets of French fries: 4 * $1.20 = $4.80\n- 5 cups of soda: 5 * $0.5 = $2.50\n- 1 platter of spaghetti: $2.70\n\nTotal cost: $15 + $4.80 + $2.50 + $2.70 = $25\n\nTo simplify the total cost, I can round the costs of the items to two decimal places:\n\n- 5 piec

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


["Upon reviewing my previous answer, I noticed a few potential issues:\n\n1. I didn't consider that the spaghetti platter might be an error, as it seems out of place among the other items (hamburgers, fries, and soda). However, I didn't have enough information to confirm this.\n2. I didn't check if the prices of the items are consistent or if there are any other items that were not mentioned (like a bill for drinks, tax, or gratuity).\n3. I didn't consider that the friends might have ordered additional items or upgrades that weren't mentioned.\n\nHowever, assuming the spaghetti platter is indeed a valid item, my calculation seems correct. If there are any other issues or inconsistencies, they would need to be addressed before I could provide a revised answer.\n\nTo confirm, could you please provide more information about the order, such as whether the spaghetti platter is a valid item or if there are any other discrepancies?", "Upon reviewing my previous answer, I found the following i

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


["Upon reviewing my previous answer, I noticed the following potential issues:\n\n1. I didn't consider rounding errors: When calculating the total bill, I used exact numbers, but in real-life scenarios, prices might be rounded to the nearest cent. For example, the hamburger cost $3, not $3.01 or $3.00. This might not be a significant issue in this case, but it's worth considering.\n\n2. I didn't check for any potential calculation errors: When adding up the costs, I didn't double-check the calculations. For instance, I didn't verify if the calculations were correct before adding the final cost of the spaghetti to the total bill.\n\nTo address these concerns, let's redo the calculations with more precise numbers:\n\n5 pieces of hamburger cost 5 x $3.00 = $15.00.\n4 sets of French fries cost 4 x $1.20 = $4.80.\n5 cups of soda cost 5 x $0.50 = $2.50.\n1 platter of spaghetti costs $2.70.\n\nNow, let's add up the costs:\n$15.00 + $4.80 = $19.80\n$19.80 + $2.50 = $22.30\n$22.30 + $2.70 = $25

In [14]:

for initial_response in initial_responses:
    ir_chat = {"role": "assistant", "content": initial_response}
    for feedbacks in feedbackss:
        for feedback in feedbacks:
            feedback_chat = {"role": "user", "content": feedback_prompt}
            feedback = {"role": "assistant", "content": feedback}
            revision_chat = {"role": "user", "content": refine_prompt}
            revision = inference([chat+[ir_chat, feedback_chat, feedback, revision_chat] for _ in range(8)], model, tokenizer, device)
            print(revision)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['To find the total cost, we need to calculate the cost of each item and add them together.\n\n- 5 pieces of hamburger cost 5 x $3 = $15.\n- 4 sets of French fries cost 4 x $1.20 = $4.80.\n- 5 cups of soda cost 5 x $0.5 = $2.50.\n- 1 platter of spaghetti costs $2.70.\n\nTotal cost = $15 + $4.80 + $2.50 + $2.70 = $25.\n\nHowever, this includes the full cost of the spaghetti. To find the correct total cost per friend, we need to split the cost of the spaghetti among the 5 friends.\n\nCost of spaghetti per friend = $2.70 ÷ 5 = $0.54.\n\nTotal cost per friend = $15 + $4.80 + $2.50 + $0.54 = $23.04.\n\nEach friend will pay $23.04.', "To find the total cost, we need to calculate the cost of each item and add them together.\n\n- 5 pieces of hamburger cost 5 x $3 = $15.\n- 4 sets of French fries cost 4 x $1.20 = $4.80.\n- 5 cups of soda cost 5 x $0.5 = $2.50.\n- 1 platter of spaghetti costs $2.70.\n\nTotal cost = $15 + $4.80 + $2.50 + $2.70 = $25.\n\nSince there are 5 friends, they will split 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['To find the total cost, we need to calculate the cost of each item and add them together.\n\n- 5 pieces of hamburger cost 5 x $3 = $15.\n- 4 sets of French fries cost 4 x $1.20 = $4.80.\n- 5 cups of soda cost 5 x $0.5 = $2.50.\n- 1 platter of spaghetti costs $2.70.\n\nTotal cost = $15 + $4.80 + $2.50 + $2.70 = $25.\n\nSince there are 5 friends, they will split the bill equally. To find out how much each friend will pay, we divide the total cost by 5:\n\n$25 ÷ 5 = $5.\n\nHowever, I want to clarify that the friends are paying for a meal that includes the spaghetti, which is a separate item from the other food. Therefore, the correct answer is $5.\n\n**boxed{5}**', 'To find the total cost, we need to calculate the cost of each item and add them together.\n\n- 5 pieces of hamburger cost 5 x $3 = $15.\n- 4 sets of French fries cost 4 x $1.20 = $4.80.\n- 5 cups of soda cost 5 x $0.5 = $2.50.\n- 1 platter of spaghetti costs $2.70.\n\nTotal cost = $15 + $4.80 + $2.50 + $2.70 = $25.\n\nSince 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['To find the total cost, we need to calculate the cost of each item and add them up.\n\n- 5 pieces of hamburger: 5 x $3 = $15\n- 4 sets of French fries: 4 x $1.20 = $4.80\n- 5 cups of soda: 5 x $0.5 = $2.50\n- 1 platter of spaghetti: $2.70\n\nTotal cost of hamburgers, French fries, and soda: $15 + $4.80 + $2.50 = $22.30\nTotal cost including spaghetti platter: $22.30 + $2.70 = $25\n\nSince there are 5 friends, they will split the spaghetti platter as well. To calculate the cost of one serving of spaghetti, we need to divide the total cost of the spaghetti platter by 5.\n\nCost of one serving of spaghetti: $2.70 ÷ 5 = $0.54\n\nNow, we can calculate the total bill:\n\n$22.30 + $2.70 + $0.54 = $25.54\n\nSince the friends are splitting the total bill equally, we can divide the total bill by 5.\n\n$25.54 ÷ 5 = $5.11\n\nEach friend will pay $5.11.', 'To find the total cost, we need to calculate the cost of each item and add them up.\n\n- 5 pieces of hamburger: 5 x $3 = $15\n- 4 sets of Fren

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['To find the total cost, we need to calculate the cost of each item and add them up.\n\n- 5 pieces of hamburger: 5 x $3.00 = $15.00\n- 4 sets of French fries: 4 x $1.20 = $4.80\n- 5 cups of soda: 5 x $0.50 = $2.50\n- 1 platter of spaghetti: $2.70\n\nTotal cost: $15.00 + $4.80 = $19.80, then $19.80 + $2.50 = $22.30, and finally $22.30 + $2.70 = $25.00\n\nSince there are 5 friends, they will split the bill equally.\n\n$25.00 ÷ 5 = $5.00\n\nEach friend will pay $5.00.', 'To find the total cost, we need to calculate the cost of each item and add them up.\n\n- 5 pieces of hamburger: 5 x $3 = $15\n- 4 sets of French fries: 4 x $1.20 = $4.80\n- 5 cups of soda: 5 x $0.5 = $2.50\n- 1 platter of spaghetti: $2.70\n\nTotal cost: $15 + $4.80 + $2.50 + $2.70 = $25.00\n\nSince there are 5 friends, they will split the bill equally. \n\n$25.00 ÷ 5 = $5.00\n\nEach friend will pay $5.00.', 'To find the total cost, we need to calculate the cost of each item and add them up.\n\n- 5 pieces of hamburger: 5 


KeyboardInterrupt

